# Technical Analysis

Roboquant comes out of box with support for technical analysis. It does this by having dedicate support for two popular technical analysis libraries:

. TaLib: contains over 150 technical indicators. These are indicators ranging from indicators such as ADX, MACD, RSI, Stochastic, Bollinger Bands to Candlestick pattern recognition. TA-Lib can be difficult to use due to its API, but roboquant makes it easy to use it in by wrapping it in conveniece access library. 

. Ta4j: contains over 130 technical indicators and has a flexible engine to define your own trading rules.

In [ ]:
%use @http://roboquant.org/roboquant.json
Welcome()

# TaLib based Strategy
We start by defining the  strategy using TALib indicators we want to test in combination with the `TaLibStrategy` class. There are three steps:

1) Create an instance of `TaLibStrategy` and define how much history it should maintain before invoking the indicators. If you choose this value too low, certain indicators will fail and an exception will be thrown. For example if you want a moving average over 20 days, the history should be at least 20. If you choose the histoy value too large, it will take longer before your strategy starts to generate signals since it will wait untill it has enough history. For many of the technical indicators like candle stick patterns, 15 is sufficient (also the default value).

2) Define what you logic to use for generating BUY signals. This can be a combination of indicators and you can also include regular Kotlin code. The only restriction is that the last statement has to be a boolean; true if you want to BUY, false otherwise.

3) Define what you logic to use for generating SELL signals. The same rules and conditions apply as for the BUY method.

Please note, that although not common practice, the framework doesn't stop you from using bullish indicators to generate a SELL signal and bearish indicators to generate a BUY signal. 

In [ ]:
// Define a strategy with maximum of 15 days of history. It is important that these are enough days to calculate the various indicators.
val strategy = TaLibStrategy(15)

// We want to generate a BUY signal if we detect either one of the following wto candlestick patternsL: Morningstar or 3WhiteSoldiers
strategy.buy {
    cdlMorningStar(it) || cdl3WhiteSoldiers(it)
}

// We want to generate a SELL signal if we detect the candlestick pattern Two Crows and also observe that the 5 day weighted average is below the 9 day weighted average 
strategy.sell {
    cdl2Crows(it) && ema(it.close, 5) < ema(it.close, 9)
}


Now we can use this strategy just like any other strategy. In this case we use the S&P500 feed.

In [ ]:
val roboquant =  Roboquant(strategy, AccountSummary())

val feed = AvroFeed.sp500()
roboquant.run(feed)

Now lets see what the outcome is. Spoiler alert, this strategy won't make you rich ;) 

In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
account.portfolio.summary()

Let find out which trade generated most profit (realized PnL) and plot the price and all the trades for that asset

In [ ]:
val asset = account.trades.maxByOrNull { it.pnl.value }!!.asset
PriceBarChart(feed, asset, account.trades)

In [ ]:
val metricData = roboquant.logger.getMetric("account.equity")
MetricChart(metricData)

## Ta4j based Strategy

We'll now will define a strategy using indicators and rules found in the `ta4j` library and the roboquant `Ta4jStrategy` strategy class. It allows to define flexible rules to generate BUY or SELL signals for assets. 

You can find out more on using `ta4j` on the wiki site  https://ta4j.github.io/ta4j-wiki/Trading-strategies.html or visit the `ta4j` Discord server. Please note that indicators and rules from `ta4j` can be freely used and mixed, but defining strategies and back tests are done using roboquant constructs like the `Ta4jStrategy`.   

The main steps to follow are:

1. Import the ta4j packages that you want to use (in future releases of roboquant this might be done automatically) 
2. Create a `Ta4jStrategy` instance and (optional) indicate how much history we want to track
3. Define the buying and selling rules
4. Run a back test and evaluate the results (this is not specific to `ta4j` based stategies)

An important difference with `ta4j` back testing and `roboquant` is that in roboqaunt by default the buying and selling rules apply to all assets found in a feed. So in the below example the defined strategy is applied to all the S&P 500 stocks and not just a single stock. 

In [ ]:
import org.ta4j.core.indicators.*
import org.ta4j.core.rules.*
import org.ta4j.core.indicators.helpers.*

In [ ]:
val strategy = Ta4jStrategy(maxBarCount = 30)

strategy.buy { series ->
    val closePrice = ClosePriceIndicator(series)
    val shortSma = SMAIndicator(closePrice, 5)
    val longSma = SMAIndicator(closePrice, 30)
    CrossedUpIndicatorRule(shortSma, longSma)
}

strategy.sell { series ->
    val closePrice = ClosePriceIndicator(series)
    val shortSma = SMAIndicator(closePrice, 5)
    val longSma = SMAIndicator(closePrice, 30)
    CrossedDownIndicatorRule(shortSma, longSma)
        .or(StopLossRule(closePrice, 3.0))
        .or(StopGainRule(closePrice, 2.0))
}

In [ ]:
val roboquant =  Roboquant(strategy, AccountSummary())
val feed = AvroFeed.sp500()
roboquant.run(feed)

In [ ]:
val metricData = roboquant.logger.getMetric("account.equity")
MetricChart(metricData)